In [122]:
from Bio import SeqIO,bgzf,pairwise2
import gzip
from Bio import 
from Bio.pairwise2 import format_alignment
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import SingleLetterAlphabet


# with bgzf.BgzfWriter("test.fastq.bgz", "wb") as outgz:
#     SeqIO.write(sequences=records, handle=outgz, format="fastq")

In [233]:
def fastq_overlap_merge(read1,read2):
    # find overlap
    alignments = pairwise2.align.localms(read1.seq, read2.seq.reverse_complement(),1,-0.1,-2,-1)
    match_begin=alignments[0][3]
    match_end=alignments[0][4]
    #locate mismatch
    mismatch=[]
    for p in range(0,match_end-match_begin):
        if read1.seq[match_begin+p] != read2.seq.reverse_complement()[p]:
            ss=[(read1.letter_annotations["phred_quality"][match_begin+p],read1.seq[match_begin+p]),(read2.letter_annotations["phred_quality"][-p],r2[k].seq.reverse_complement()[p])]
            ss.sort(key=lambda tup: tup[0], reverse= True)
            mismatch.append((match_begin+p,ss[0]))
        
    s=read1+read2[:match_begin-match_end].reverse_complement()
    s.name=read1.name
    s.id=read1.id
    s.description= read1.description + " merged with " +read2.description.split()[-1]
    phred_quality=read1.letter_annotations["phred_quality"][:match_begin]\
    +[max(a,b) for (a,b) in \
      zip(read1.letter_annotations["phred_quality"][match_begin:], \
          read2.letter_annotations["phred_quality"][::-1][:(match_end-match_begin)])]\
    + read2.letter_annotations["phred_quality"][::-1][(match_end-match_begin):]
    s.letter_annotations["phred_quality"]=phred_quality
    
    # replace with the best base
    if len(mismatch)>0:
        for pp, (qu, bb) in mismatch:
            s.seq=s.seq[:pp] + bb + s.seq[pp+1:]
    return s


In [4]:
r1="../data/AAVS912-2-WTA-MECOM-AMP_B5_S689_L001_R1_001.fastq.gz"
r2="../data/AAVS912-2-WTA-MECOM-AMP_B5_S689_L001_R2_001.fastq.gz"

In [11]:
with gzip.open(r1, "rt") as handle:
    r1=SeqIO.to_dict(SeqIO.parse(handle, "fastq"))


In [9]:
with gzip.open(r2, "rt") as handle:
    r2=SeqIO.to_dict(SeqIO.parse(handle, "fastq"))


In [250]:
un_merged_r1=dict()
un_merged_r2=dict()
merged_records=dict()
for k, i in r1.items():
#     print (k,i.seq, i.letter_annotations["phred_quality"])
#     print (r2[k].seq,r2[k].letter_annotations["phred_quality"])
    try:
        merged_records[k]=fastq_overlap_merge(i,r2[k])
    except:
        un_merged_r1[k]=i
        un_merged_r2[k]=r2[k]
with open('../data/un_merged_r1.fastq', 'w') as handle:
    SeqIO.write(un_merged_r1.values(), handle, 'fastq')     
with open('../data/un_merged_r2.fastq', 'w') as handle:
    SeqIO.write(un_merged_r2.values(), handle, 'fastq')  
with open('../data/merged.fastq', 'w') as handle:
    SeqIO.write(merged_records.values(), handle, 'fastq')  